In [ ]:
!pip install autokeras
!pip install keras-tuner

     |████████████████████████████████| 166 kB 5.2 MB/s 
     |████████████████████████████████| 454.3 MB 16 kB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 1.2 MB 48.7 MB/s 
     |████████████████████████████████| 462 kB 47.3 MB/s 
     |████████████████████████████████| 4.0 MB 43.5 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.40.0
    Uninstalling grpcio-1.40.0:
      Successfully uninstalled grpcio-1.40.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak
import re
import nltk
from nltk.corpus import stopwords
import time
import datetime

In [4]:
cd '/content/drive/MyDrive/DCC/data'

/content/drive/MyDrive/DCC/data


In [5]:
train = pd.read_csv('./nlp/train.csv', encoding = 'utf-8')
test = pd.read_csv('./nlp/test_x.csv', encoding = 'utf-8')
submission = pd.read_csv('./nlp/sample_submission.csv', encoding = 'utf-8')

In [ ]:
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [6]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)
test['text']=test['text'].apply(alpha_num)

In [7]:
nltk.download("stopwords")
#불용어 처리
def stop_words(text):
    r = []
    stop_words = list(set(stopwords.words('english')))
    word = text.split(' ')
    for w in word:
      if w not in stop_words:
        r.append(w)    
    return ' '.join(r)

train['text_stopword'] = train['text'].apply(stop_words)
test['text_stopword']=test['text'].apply(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
X_train = train.text_stopword.values
y_train = train.author.values
test = test.text_stopword.values

>  Vanilla RNN -> Transformer -> Bert 순으로 학습함

*파이썬 버전 3.6이 되어야 bert가 되는 것 같음*

In [9]:
start = time.time()

clf = ak.TextClassifier(
    multi_label=True,
    overwrite=True,
    max_trials=2)

clf.fit(X_train, y_train, epochs=5)

model = clf.export_model()
sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)

Trial 2 Complete [00h 11m 41s]
val_loss: 0.2365383356809616

Best val_loss So Far: 0.2365383356809616
Total elapsed time: 00h 36m 58s
INFO:tensorflow:Oracle triggered exit
Epoch 1/5
1715/1715 [==============================] - 162s 93ms/step - loss: 0.4260 - accuracy: 0.4626
Epoch 2/5
1715/1715 [==============================] - 178s 104ms/step - loss: 0.2790 - accuracy: 0.6995
Epoch 3/5
1715/1715 [==============================] - 156s 91ms/step - loss: 0.2316 - accuracy: 0.7549
Epoch 4/5
1715/1715 [==============================] - 155s 90ms/step - loss: 0.2064 - accuracy: 0.7874
Epoch 5/5
1715/1715 [==============================] - 155s 90ms/step - loss: 0.1912 - accuracy: 0.8020
INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


0:52:35


In [ ]:
# #ngram등 벡터화? 가능
# # https://colab.research.google.com/github/keras-team/autokeras/blob/master/docs/ipynb/text_classification.ipynb#scrollTo=SwuLZ_bdMEjH 참고
# input_node = ak.TextInput()
# output_node = ak.TextBlock(block_type="ngram")(input_node)
# output_node = ak.ClassificationHead()(output_node)
# clf = ak.AutoModel(
#     inputs=input_node, outputs=output_node, overwrite=True, max_trials=3
# )
# clf.fit(X_train, y_train, epochs=2)

In [10]:
pred = model.predict(test)
submission[['0','1','2','3','4']] = pred
submission.to_csv('./nlp/submission_autokeras.csv', index = False, encoding = 'utf-8')